# <pre> Interactive Geospatial Dashboard using Streamlit

In [ ]:
!pip install streamlit
!pip install pyngrok

In [3]:
from pyngrok import ngrok

!ngrok authtoken your_autorization_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
url = ngrok.connect(8501, 'http')
print(f'Access your geospatial dashboard at {url}')

Access your geospatial dashboard at NgrokTunnel: "https://51c1-104-199-208-211.ngrok-free.app" -> "http://localhost:8501"


WRITING STREAMLIT CODE

## 1. Page Configuration

In [6]:
!streamlit run app.py &>/content/logs.txt &

In [80]:
%%writefile app.py

import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

st.set_page_config(
    page_title = 'Geospatial Dashboard',
    page_icon = '	:globe_with_meridians:',
    layout = 'wide'
)

Overwriting app.py


In [81]:
%%writefile -a app.py
df = pd.read_csv('/content/world_population.csv')

Appending to app.py


## 2. Creating the Sidebars

In [82]:
%%writefile -a app.py

with st.sidebar:
  st.title(':globe_with_meridians: Geospatial Dashboard')

  # A. CONTINENT
  continent_lst = list(df['Continent'].unique())
  select_continent = st.multiselect('Select a Continent', continent_lst)
  if select_continent:
    df = df[df['Continent'].isin(select_continent)]
  else:
    df = df.copy()

  # B. Country
  country_lst = list(df['Country/Territory'].unique())
  select_country = st.multiselect('Select a Country', country_lst)
  if select_country:
    df = df[df['Country/Territory'].isin(select_country)]
  else:
    df = df.copy()

Appending to app.py


## 3. Choropleth map for Population

In [83]:
%%writefile -a app.py
st.subheader('Choropleth Map : Population')
fig1 = px.choropleth(df.dropna(),
                     locations = 'Country/Territory',
                     locationmode = 'country names',
                     color = 'World Population Percentage',
                     hover_name = 'Country/Territory',
                     hover_data = ['World Population Percentage', 'Area (km²)'],
                     color_continuous_scale = 'Inferno')

fig1.update_geos(
    showcoastlines = True,
    coastlinecolor = 'white',
    showland = True,
    landcolor = 'gray',
    showocean = True,
    oceancolor = 'black',
    showlakes = True,
    lakecolor = 'black',
    bgcolor = 'black'
)

st.plotly_chart(fig1, use_container_width = True)

Appending to app.py


## 4. Metrics Card

In [84]:
%%writefile -a app.py
block1, block2 = st.columns((2))

Appending to app.py


In [85]:
%%writefile -a app.py
#css for metric card
st.markdown("""
  <style>
  .card{
    border-radius: 5px;
    padding: 5px;
    margin:100px 0px 0px -50px;
    color: white;
    font-size:30px;
    text-align:center;
  }
  .value{
    font-size:48px;
    font-weight:bold;
  }
  .change-positive{
    color:yellow;
    font-size:24px;
    font-weight:bold;
  }
  .change-negative{
    color:red;
    font-size:24px;
    font-weight:bold;
  }
  </style>
""", unsafe_allow_html = True)

Appending to app.py


In [86]:
%%writefile -a app.py

with block1:
  st.subheader('Growth Rate :chart_with_upwards_trend:')
  if not select_country:
    country_data = df.iloc[0]
  else:
    country_data = df[df['Country/Territory'] == select_country[0]].iloc[0]

  #function to compute growth percentage
  def calculate_growth(previous, current):
    return ((current - previous) / previous) * 100

  #calculate growth
  country_growth = calculate_growth(country_data['2000 Population'], country_data['2022 Population'])

  #define card creation function
  def create_metric_card(country_name, current_population, growth):
    if growth > 0:
      growth_class = 'change-positive'
      growth_symbol = '↑'
    else:
      growth_class = 'change-negative'
      growth_symbol = '↓'
    return f"""
    <div class = 'card'>
    <div>{country_name}</div>
    <div class='value'>{current_population:.1f} M</div>
    <div class = "{growth_class}">{growth_symbol} {abs(growth):.2f}%</div?]>
    """
  st.markdown(create_metric_card(country_data['Country/Territory'], country_data['2022 Population'], country_growth),
            unsafe_allow_html = True)

Appending to app.py


## 5. Donut Chart

In [87]:
%%writefile -a app.py
with block2:
  st.subheader('Donut Chart : Population by Continent')
  df_continent_pop = df.groupby('Continent')['2022 Population'].sum().reset_index()
  fig2 = px.pie(df_continent_pop,
                names = 'Continent',
                values = '2022 Population',
                hole = 0.4,
                title = 'Population distribution by continent (2022)')
  st.plotly_chart(fig2, use_container_width = True)

Appending to app.py


## 6. Heatmap

In [88]:
%%writefile -a app.py
st.subheader('Heatmap : Population Growth Over Years')
population_cols = ['1970 Population', '1980 Population', '1990 Population', '2000 Population', '2010 Population']
df_heatmap = df[population_cols]

fig3 = go.Figure(data = go.Heatmap(
                                    z = df_heatmap.T,
                                    x = df['Country/Territory'],
                                    y = population_cols,
                                    colorscale = 'Plasma'
))
st.plotly_chart(fig3, use_container_width = True)

Appending to app.py
